<a href="https://colab.research.google.com/github/iam-abbas/Reddit-Stock-Trends/blob/main/Reddit_Stock_Trends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trending Stocks on Sub-Reddits

### Installing Reddit and Yahoo Finance API wrappers

In [93]:
pip install yfinance praw==7.0

### Imports

In [94]:
import numpy as np
import pandas as pd
import yfinance as yf
import re
import praw

### Stop words and Blacklist containing jargon/acronyms 
I added GME and AMC in there lmao, tired of seeing those

In [95]:
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'must', 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [96]:
block_words = ["DIP", "", "$", "RH", "YOLO", "PORN", "BEST", "MOON", "HOLD", "FAKE", "WISH", "USD", "EV", "MARK", "RELAX", "LOL", "LMAO", "LMFAO", "EPS", "DCF", "NYSE", "FTSE", "APE", "CEO", "CTO", "FUD", "DD", "AM", "PM", "FDD", "EDIT", "TA", "UK", "AMC", "GME"]

### Simple Regex to get tickers from text

In [97]:
def extract_ticker(body):
  ticks = re.findall("[$][A-Za-z]*|[A-Z][A-Z]{1,}", str(body))
  res = set()
  for item in ticks:
    if item not in block_words and item.lower() not in stop_words and item:
      try:
        tic = item.replace("$", "").upper()
        res.add(tic)
      except:
        pass
  return res
   

### Reddit API setup

In [98]:
reddit = praw.Reddit(
    client_id="9Aq-wTeGLJBKsQ", 
    client_secret="EclWNx5qOyIZLiRkd10Oln0iNPUXvQ", 
    user_agent="ScrapeStocks"
)

### Scrape subreddits `r/robinhoodpennystocks` and `r/pennystocks`
Current it does fetch a lot of additional data like upvotes, comments, awards etc but not using anything apart from title for now

In [99]:
_posts = []

new_bets = reddit.subreddit("robinhoodpennystocks+pennystocks").new(limit=None)
new_bets

for post in new_bets:
    _posts.append(
        [
            post.id,
            post.title,
            post.score,
            post.num_comments,
            post.upvote_ratio,
            post.total_awards_received,
        ]
    )

# create a dataframe
_posts = pd.DataFrame(
    _posts,
    columns=[
        "id",
        "title",
        "score",
        "comments",
        "upvote_ratio",
        "total_awards",
    ],
)

In [53]:
_posts

,id,title,score,comments,upvote_ratio,total_awards
0,ldvzrw,"CBDD, I believe would be a nice long term hold...",1,1,1.00,0
1,ldvzpy,When I joined r/PennyStocks I had a lot less Z...,1,1,1.00,0
2,ldvx6v,Congrats who held $OCGN since it was a penny s...,3,2,1.00,0
3,ldv9d7,Thank you High Tide. This will be a great year...,9,8,1.00,0
4,ldv72k,Getting close to my 10k goal!,11,13,0.82,0
...,...,...,...,...,...,...
1913,l6qk3f,STOP buying into these random redditors' ideas...,20,32,0.73,1
1914,l6qgkr,I can tell you POTATO_IN_MY_ASS had nothing to...,149,9,0.99,0
1915,l6qfbp,Why did (EXPR) moon today,3,3,0.67,0
1916,l6q9xd,"Gamestop, Robinhood and Reddit is a lethal com...",7,6,0.71,0


### Extract tickers from all titles and creae a new columns

In [54]:
_posts["Tickers"] = _posts.title.apply(extract_ticker)

In [55]:
ticker_sets = _posts.Tickers.to_list()

### Count number of occurances of the Ticker and verify id the Ticker exists
This kinda works like a hashmap

In [56]:
counts ={}

In [57]:
for s in ticker_sets:
  for tic in s:
    if tic in counts:
      counts[tic] += 1
    else:
      counts[tic] = 1

In [58]:
verified_tics = {}

In [59]:
for tic in counts:
  try:
    if counts[tic] > 3:
      print(tic, end=", ")
      yf.Ticker(tic).info
      verified_tics[tic] = counts[tic]
    else: 
      continue
  except:
    pass

CBDD
OCGN
COVID
TSNP
RLFTF
SNDL
TNXP
CTRM
AEZS
LOUNGE
SNPW
CBBT
RIGL
OTC
ZOM
FDA
BLSP
ATOS
TPTW
HITIF
PTE
PVDG
ILUS
NAKD
OTLK
ENERGY
RSSV
MLFB
CBD
CPSH
US
CIDM
TXMD
SSFT
SENS
CTXR
NASDAQ
BOTY
PR
TSX
REI
NOK
SEC
INKW
ACRX
IDEX
AREC
DSS
BNGO
ALNA
AGTC
SRGA
PT
TYME
TAKOF
ALPP
CRBP
HUMBL
BUY
UAPC
GTE
TRCH
TTOO
RYCEY
DSGT
SOS
NAK
COPRF
ONTX
REZZF
LETS
NSPR
HUGE
STOCK
OTCM
SLS
OGI
OZSC
VPER
HCMC
PSA
INND
ATVK
VYNE
XSPA
ABML
WSB
JAGX
GNUS
BB


In [60]:
verified_tics

{'ABML': 9,
 'ACRX': 15,
 'AEZS': 11,
 'AGTC': 6,
 'ALNA': 4,
 'ALPP': 4,
 'AREC': 4,
 'ATOS': 71,
 'ATVK': 4,
 'BB': 6,
 'BLSP': 6,
 'BNGO': 5,
 'BOTY': 5,
 'BUY': 8,
 'CBBT': 9,
 'CBD': 4,
 'CBDD': 4,
 'CIDM': 14,
 'COPRF': 4,
 'CPSH': 4,
 'CRBP': 5,
 'CTRM': 18,
 'CTXR': 8,
 'DSGT': 4,
 'DSS': 4,
 'GNUS': 4,
 'GTE': 4,
 'HCMC': 22,
 'HITIF': 7,
 'HUGE': 5,
 'IDEX': 8,
 'ILUS': 6,
 'INKW': 10,
 'INND': 4,
 'JAGX': 4,
 'MLFB': 5,
 'NAK': 7,
 'NAKD': 16,
 'NOK': 32,
 'NSPR': 7,
 'OCGN': 13,
 'OGI': 4,
 'ONTX': 5,
 'OTCM': 5,
 'OTLK': 4,
 'OZSC': 7,
 'PSA': 4,
 'PT': 6,
 'PTE': 11,
 'PVDG': 9,
 'REI': 19,
 'REZZF': 6,
 'RIGL': 4,
 'RLFTF': 4,
 'RSSV': 5,
 'RYCEY': 8,
 'SENS': 13,
 'SLS': 5,
 'SNDL': 49,
 'SNPW': 13,
 'SOS': 5,
 'SRGA': 4,
 'SSFT': 4,
 'TAKOF': 4,
 'TNXP': 11,
 'TPTW': 5,
 'TRCH': 7,
 'TSNP': 12,
 'TTOO': 5,
 'TXMD': 8,
 'TYME': 5,
 'UAPC': 5,
 'VPER': 4,
 'VYNE': 7,
 'XSPA': 6,
 'ZOM': 19}

### Create Datable of just mentions

In [61]:
tick_df = pd.DataFrame(verified_tics.items(), columns=["Ticker", "Mentions"])

In [62]:
tick_df

,Ticker,Mentions
0,CBDD,4
1,OCGN,13
2,TSNP,12
3,RLFTF,4
4,SNDL,49
...,...,...
71,XSPA,6
72,ABML,9
73,JAGX,4
74,GNUS,4


### Sort

In [63]:
tick_df.sort_values(by=["Mentions"], inplace=True, ascending=False)
tick_df.reset_index(inplace=True, drop=True)
tick_df

,Ticker,Mentions
0,ATOS,71
1,SNDL,49
2,NOK,32
3,HCMC,22
4,ZOM,19
...,...,...
71,DSGT,4
72,SRGA,4
73,TAKOF,4
74,GTE,4


### Use Yahoo Finance API to get the relavent data

In [64]:
def calculate_change(start, end):
  return round(((end - start)/start)*100, 2)

In [65]:
def get_change(ticker, period="1d"):
   return calculate_change(yf.Ticker(ticker).history(period)["Open"].to_list()[0], yf.Ticker(ticker).history(period)["Close"].to_list()[-1])
  

In [79]:
tick_df.dropna(axis=1)
n25_df = tick_df.head(25)

In [80]:
n25_df["Name"] = n25_df.Ticker.apply(lambda x: yf.Ticker(x).info["longName"])
n25_df["Bid"] = n25_df.Ticker.apply(lambda x: yf.Ticker(x).info["previousClose"])
n25_df["5d Low"] = n25_df.Ticker.apply(lambda x: min(yf.Ticker(x).history(period="5d")['Low'].to_list()))
n25_df["5d High"] = n25_df.Ticker.apply(lambda x: max(yf.Ticker(x).history(period="5d")['High'].to_list()))
n25_df["1d Change (%)"] = n25_df.Ticker.apply(lambda x: get_change(x))
n25_df["5d Change (%)"] = n25_df.Ticker.apply(lambda x: get_change(x, "5d"))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [81]:
n25_df["1mo Change (%)"] = n25_df.Ticker.apply(lambda x: get_change(x, "1mo"))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [82]:
n25_df

,Ticker,Mentions,Name,Bid,5d Low,5d High,1d Change (%),5d Change (%),1mo Change (%)
0,ATOS,71,"Atossa Therapeutics, Inc.",4.33000,2.15000,4.9000,-2.99,64.14,332.22
1,SNDL,49,Sundial Growers Inc.,1.15000,0.90200,1.3300,-5.04,13.00,71.73
2,NOK,32,Nokia Corporation,4.37000,4.20000,4.9900,-2.54,-15.09,5.24
3,HCMC,22,Healthier Choices Management Corp.,0.00170,0.00100,0.0050,-11.11,14.29,1500.00
4,ZOM,19,Zomedica Corp.,1.50000,0.95000,2.0100,12.35,97.52,365.85
5,REI,19,"Ring Energy, Inc.",1.37000,1.07000,1.7500,13.99,41.74,117.33
6,CTRM,18,Castor Maritime Inc.,0.67360,0.61500,0.7750,14.02,2.30,221.28
7,NAKD,16,Naked Brand Group Limited,1.17000,0.74000,1.9000,-5.98,-41.49,223.53
8,ACRX,15,"AcelRx Pharmaceuticals, Inc.",2.32000,2.03000,2.4200,-3.73,3.57,71.85
9,CIDM,14,Cinedigm Corp.,1.87000,1.23000,1.9800,-7.22,39.53,146.58


In [83]:
n25_df.rename(columns={'Bid': 'Price - 2/5'}, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [85]:
n25_df["Industry"] = n25_df.Ticker.apply(lambda x: yf.Ticker(x).info['industry'])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [86]:
n25_df

,Ticker,Mentions,Name,Price - 2/5,5d Low,5d High,1d Change (%),5d Change (%),1mo Change (%),Industry
0,ATOS,71,"Atossa Therapeutics, Inc.",4.33000,2.15000,4.9000,-2.99,64.14,332.22,Biotechnology
1,SNDL,49,Sundial Growers Inc.,1.15000,0.90200,1.3300,-5.04,13.00,71.73,Drug Manufacturers—Specialty & Generic
2,NOK,32,Nokia Corporation,4.37000,4.20000,4.9900,-2.54,-15.09,5.24,Communication Equipment
3,HCMC,22,Healthier Choices Management Corp.,0.00170,0.00100,0.0050,-11.11,14.29,1500.00,Grocery Stores
4,ZOM,19,Zomedica Corp.,1.50000,0.95000,2.0100,12.35,97.52,365.85,Drug Manufacturers—Specialty & Generic
5,REI,19,"Ring Energy, Inc.",1.37000,1.07000,1.7500,13.99,41.74,117.33,Oil & Gas E&P
6,CTRM,18,Castor Maritime Inc.,0.67360,0.61500,0.7750,14.02,2.30,221.28,Marine Shipping
7,NAKD,16,Naked Brand Group Limited,1.17000,0.74000,1.9000,-5.98,-41.49,223.53,Apparel Manufacturing
8,ACRX,15,"AcelRx Pharmaceuticals, Inc.",2.32000,2.03000,2.4200,-3.73,3.57,71.85,Drug Manufacturers—Specialty & Generic
9,CIDM,14,Cinedigm Corp.,1.87000,1.23000,1.9800,-7.22,39.53,146.58,Entertainment
